# Getting familiar with the electrophysiology (Ephys) pipeline

In this notebook, we will take a tour of the Moser group electrophysiology (ephys) pipeline, taking a close look at how each table is defined and relate to each other to represent various experimental information.

But before we begin, let's import a couple essential packages, in particular importing DataJoint (`datajoint`) package, and establish connection to the Moser lab database where the entire pipeline resides.

In [1]:
import datajoint as dj

In [2]:
dj.config['display.limit'] = 10

If you have followed all the setups for Python environment, DataJoint and the Ephys pipeline as described in [Zero to DataJoint Wiki page](https://github.com/kavli-ntnu/dj-elphys/wiki/Zero-to-datajoint), you should be able to just run the next line, importing pipeline definitions as found in the `ephys` package.

In [3]:
dj.config['database.host'] = 'datajoint.it.ntnu.no'

In [4]:
dj.conn()

Connecting thinh@datajoint.it.ntnu.no:3306


DataJoint connection (connected) thinh@datajoint.it.ntnu.no:3306

In [5]:
animal = dj.create_virtual_module('animal', 'prod_mlims_data')
reference = dj.create_virtual_module('reference', 'group_shared_reference')
acquisition = dj.create_virtual_module('acquisition', 'group_shared_acquisition')
tracking = dj.create_virtual_module('tracking', 'group_shared_tracking')
behavior = dj.create_virtual_module('behavior', 'group_shared_behavior')
ephys = dj.create_virtual_module('ephys', 'group_shared_ephys')
analysis = dj.create_virtual_module('analysis', 'group_shared_analysis')

If you experienced any issue in importing the pipeline, please refer to the [Wiki page and guide](https://github.com/kavli-ntnu/dj-elphys/wiki/Zero-to-datajoint) again or seek help from administrators.

The entire pipeline is quite elaborate and contains many parts, grouping related tables together to facilitate navigation and understanding. Rather than looking at them all at once, we will now take a look at each of these groups of tables or **schemas**, one at a time.

## Animal information

Just like any experiment cannot occur without the animal, the entire pipeline also begin with capturing information about the animal. Naturally, information pertaining to the animal can be found under the `animal` schema.

![](../images/ephys_pipeline/animal.png)

Among other things, the `animal.Animal` table contains listing of all animals in the lab, and as we will see, serve as the starting point for the entire ephys pipeline.

In [6]:
animal.Animal()

animal_id,datasource_id,animal_species,animal_name,animal_sex,animal_dob,color
0000da5d6810d149,0,mouse,85106,F,2018-10-07,None
0000df481fc1421d,0,rat,20399,F,2014-08-06,None
00016d089675e2f2,0,mouse,55885,M,2016-02-12,None
0003a07550faa59b,0,mouse,78224,F,2017-11-30,None
0005070177484ee2,0,rat,20259,M,2014-07-21,None
0005baadae264d7b,0,rat,24790,F,2018-04-12,None
00062843f7d3ffce,0,mouse,55391,F,None,None
00062e8c65bb14f3,0,mouse,52756,M,2015-02-27,None
0006d0993fd101e8,0,mouse,78886,F,2017-12-25,None
00080a60ab6366cd,0,mouse,83976,F,2018-08-01,None


Each animal is uniquely captured by a combination of `animal_id` and `datasource_id`. `datasource_id=0` indicates that these are animals found in MLIMS colony management system.

### Querying tables

In DataJoint, you can probe into or **query** tables to get a specific subset of data that you may be interested in. The most common type of query is **restriction**, where you'll subselect entries based on specific criteria.

#### Question 1: Find only rats

#### Question 2: Find only animals born after 2017

### Getting data out of the table

Once you form your desired query result, you can **fetch** back the data from the tables by calling `fetch` on the query result.

In [7]:
# get all information about animals born on or after 2017
data = (animal.Animal & 'animal_dob >= "2017-01-01"').fetch()

In [8]:
data

array([('0000da5d6810d149', 0, 'mouse', '85106', 'F', datetime.date(2018, 10, 7), 'None'),
       ('0003a07550faa59b', 0, 'mouse', '78224', 'F', datetime.date(2017, 11, 30), 'None'),
       ('0005baadae264d7b', 0, 'rat', '24790', 'F', datetime.date(2018, 4, 12), 'None'),
       ...,
       ('aaf70b0da8daf346', 0, 'mouse', '76008', 'M', datetime.date(2017, 8, 23), 'None'),
       ('aafda9b6a7263a17', 0, 'mouse', '86528', 'M', datetime.date(2018, 12, 21), 'None'),
       ('aafebb46548b68ea', 0, 'mouse', '81878', 'M', datetime.date(2018, 4, 20), 'None')],
      dtype=[('animal_id', 'O'), ('datasource_id', '<i8'), ('animal_species', 'O'), ('animal_name', 'O'), ('animal_sex', 'O'), ('animal_dob', 'O'), ('color', 'O')])

You can also fetch just the specific columns by specifying the column names when calling `fetch`

In [9]:
names, dobs = (animal.Animal & 'animal_dob >= "2017-01-01"').fetch('animal_name', 'animal_dob')

In [10]:
names

array(['85106', '78224', '24790', ..., '76008', '86528', '81878'],
      dtype=object)

In [11]:
dobs

array([datetime.date(2018, 10, 7), datetime.date(2017, 11, 30),
       datetime.date(2018, 4, 12), ..., datetime.date(2017, 8, 23),
       datetime.date(2018, 12, 21), datetime.date(2018, 4, 20)],
      dtype=object)

### Getting list of unique animals

You can just get the identifying information of the table by grabbing (or **fetching**) the `KEY` of the table, and then using this to **restrict** the entries.

In [12]:
keys = (animal.Animal & 'animal_dob >= "2017-01-01"').fetch('KEY')

In [13]:
keys[:5]  # first five keys

[{'animal_id': '0000da5d6810d149', 'datasource_id': 0},
 {'animal_id': '0003a07550faa59b', 'datasource_id': 0},
 {'animal_id': '0005baadae264d7b', 'datasource_id': 0},
 {'animal_id': '0006d0993fd101e8', 'datasource_id': 0},
 {'animal_id': '00080a60ab6366cd', 'datasource_id': 0}]

Get specific animal back

In [14]:
# get the first animal back
animal.Animal & keys[0]

animal_id,datasource_id,animal_species,animal_name,animal_sex,animal_dob,color
0000da5d6810d149,0,mouse,85106,F,2018-10-07,None


## Electrophysiology Session

In order to understand how the entire `ephys` pipeline is organized, it is essential to first understand what we mean by "a session".

The following figure illustrates the structure of an electrophysiology experiment session.

<img src="../images/Ephys_Session_Structure.png" alt="drawing" width="700"/>

A single experimental **session** consists of one or more of the following:
* tracking data stream (recording from animal tracker) -- referred to as **tracking** data
* electrophsyiology data stream (recording from a single probe) -- referred to as **recording** data

Within a single **session**, you can get any number of the above, with possibility of having multiple probe recordings occuring simultaneously (e.g. when recording from two or more probes at the same time).

While the experimenter is free to split any number of recordings/trackings into a desired number of sessions, an **experimental session** is typically considered to be a single coherent collection of recordings, and is also typically the basic unit for the downstream analysis.

Additionally, a single **session** may be assocciated with one or more **tasks**, usually labeling the particular behavioral task or environment that the animal is exposed to.

To represent the above complexity, we make use of multiple related tables, starting with `Session`, representing a single experimental session in the pipeline. 

An experimental `Session` can be uniquely identified by knowing the **animal** and **the exact date time** of the experiment session. This is reflected in the definition of the `Session` table.

In [15]:
acquisition.Session()

animal_id,datasource_id,session_time start time of this session,session_note
00342c6d86e97d43,0,2019-07-04 20:35:12,=BLOB=
40bfa03a7a7018cc,0,2019-09-17 19:43:06,=BLOB=
61fd2ac184c13c73,0,2017-10-02 15:54:25,=BLOB=
61fd2ac184c13c73,0,2017-10-09 16:07:29,=BLOB=
61fd2ac184c13c73,0,2017-10-13 10:32:43,=BLOB=
61fd2ac184c13c73,0,2017-11-02 15:08:37,=BLOB=
61fd2ac184c13c73,0,2017-11-17 15:17:29,=BLOB=
61fd2ac184c13c73,0,2017-11-18 15:53:07,=BLOB=
61fd2ac184c13c73,0,2017-12-04 11:50:49,=BLOB=
61fd2ac184c13c73,0,2017-12-08 16:54:35,=BLOB=


Looking at the *definition* of the table, we see that `Session` table **refers to `Animal` table** to indicate that each `Sessions` depends on an `Animal`. In other words, you cannot possibly have an experiment session without a corresponding animal.

In [16]:
acquisition.Session.describe();

-> animal.Animal
session_time         : datetime                     # start time of this session
---
session_note=null    : longblob                     



This relationship is captured by the following table **diagram**

<img src="../images/ephys_pipeline/animal_session.png" alt="drawing" width="300"/>

The power of DataJoint query shines when you start combining multiple tables in your query to ask more complex questions!

#### Question 3: Find all animals with one or more experimental session

#### Question 4: Find all animals with sessions run on or after 2018.

## Recording and Tracking

As was mentioned above, we expect to find one or more electrode probe recordings and/or tracking data stream. These information are represented by `Recording` and `Tracking` tables, respectively.

<img src="../images/ephys_pipeline/session_simple.png" alt="drawing" width="600"/>

The extended diagram now captures the relationship between `Session` and the two new tables `Recording` and `Tracking`. However, notice that `Recording` also *depends* on another table -- `ProbeInsertion`.

## Understanding recording - working with probes

In [17]:
acquisition.Recording.describe();

-> acquisition.Session
-> acquisition.ProbeInsertion
recording_time       : datetime                     # start time of this recording
---
-> reference.RecordingSystem
-> reference.ElectrodeConfig
recording_order      : smallint                     # the ordering of this recording in this session
recording_duration   : float                        # (s) duration of this recording
recording_name=""    : varchar(40)                  # name of this recording (e.g. 27032019laserSess1)



Looking at the definition of `Recording` table, we can see that it depends on two parent tables -- `acquisition.Session` and `acquisition.ProbeInsertion`. Dependency on `acquisition.Session` ensures that each recording must be associated with exactly one `Session` entry. What is the dependency on `ProbeInsertion` about? 

If you guessed that this captures exactly which electrode probe that the recording was performed on, then you guessed right! `ProbeInsertion` table actually represents the final table in a chain of tables capturing precise information about the probe that was inserted into the animal.

<img src="../images/ephys_pipeline/probe_insert.png" alt="drawing" width="450"/>

In [18]:
acquisition.ProbeInsertion.describe();

-> animal.Animal
insertion_time       : datetime                     # When this probe was inserted
---
-> reference.Probe
insertion_note=""    : varchar(1000)                # some notes



Conceptually, an entry in `ProbeInsertion` table represents a particular event of probe insertion into the animal, and this is uniquely identified by knowing on **which animal** and **when** the insertion was performed. You can also see that `ProbeInsertion` *refers* to `Probe` table, thereby indicating what `Probe` was actually inserted.

In [19]:
reference.Probe()

probe unique identifier of this probe (e.g. serial number),"probe_model nick name, or other user-friendly model name of this probe"
microdrive_86866,4-tetrode-array
tetrode_74472,4-tetrode-array
tetrode_74956,4-tetrode-array
tetrode_88068_1,4-tetrode-array
tetrode_88068_2,4-tetrode-array
microdrive_24321_1,8-tetrode-array
microdrive_24321_2,8-tetrode-array
tetrode_32c,8-tetrode-array
Neuropixel_1,neuropixels_1.0
Neuropixel_2,neuropixels_1.0


In [20]:
reference.Probe.describe();

# represent a physical probe
probe                : varchar(32)                  # unique identifier of this probe (e.g. serial number)
---
-> reference.ProbeModel



An entry in `Probe` table refers to a specific *physical instance* of a probe, with some string of characters used to uniquely identify the probe (e.g. name or serial number of the probe). Again, `Probe` *refers* to `ProbeModel` table, that now captures the **model** of the probe, such as neuropixel probe version 1.0 or 4 probe tetrode array.

In [21]:
reference.ProbeModel()

"probe_model nick name, or other user-friendly model name of this probe",probe_type
neuropixels_1.0,neuropixels
4-tetrode-array,tetrode_array
8-tetrode-array,tetrode_array


In [22]:
reference.ProbeModel.describe();

# represent a probe model
probe_model          : varchar(32)                  # nick name, or other user-friendly model name of this probe
---
-> reference.ProbeType



Finally, each `ProbeModel` *refers* to the type of probe, which is a more general grouping of probe classifiation, such as neuropixels or tetrode array.

In [23]:
reference.Probe * reference.ProbeModel 

probe unique identifier of this probe (e.g. serial number),"probe_model nick name, or other user-friendly model name of this probe",probe_type
Neuropixel_1,neuropixels_1.0,neuropixels
Neuropixel_2,neuropixels_1.0,neuropixels
neuropixel_25558,neuropixels_1.0,neuropixels
microdrive_86866,4-tetrode-array,tetrode_array
tetrode_74472,4-tetrode-array,tetrode_array
tetrode_74956,4-tetrode-array,tetrode_array
tetrode_88068_1,4-tetrode-array,tetrode_array
tetrode_88068_2,4-tetrode-array,tetrode_array
microdrive_24321_1,8-tetrode-array,tetrode_array
microdrive_24321_2,8-tetrode-array,tetrode_array


<img src="../images/ephys_pipeline/probe_insert.png" alt="drawing" width="450"/>

Put together, `ProbeInsertion` allows experimenters to capture the information about the probe that was inserted into an animal.

#### Question 5: Find all animals with tetrode array implants

## Recordings

Now we understand what `ProbeInsertion` table captures, let's get back to `Recording` table.

<img src="../images/ephys_pipeline/session_simple.png" alt="drawing" width="700"/>

In [24]:
acquisition.Recording.describe();

-> acquisition.Session
-> acquisition.ProbeInsertion
recording_time       : datetime                     # start time of this recording
---
-> reference.RecordingSystem
-> reference.ElectrodeConfig
recording_order      : smallint                     # the ordering of this recording in this session
recording_duration   : float                        # (s) duration of this recording
recording_name=""    : varchar(40)                  # name of this recording (e.g. 27032019laserSess1)



Notice that a entry in `Recording` is uniquely identified by a unique combination of a `Session`, `ProbeInsertion`, **and recording time**! It is the presence of this last primary key attribute that enables more than one recording to be present for each combination of `Sessions` and `ProbeInsertion`. In other words, within a `Session`, you can have more than one recordings from the same probe, as long as they are separated in time (`recording_time`).

Now you may have noticed that `Recording` also *refers* to a number of other tables. This is graphically depicted by the following diagram.

<img src="../images/ephys_pipeline/recording.png" alt="drawing" height="400"/>

`RecordingSystem`, as the name indicates, keeps track of different recroding systems.

In [25]:
reference.RecordingSystem()

recording_system
axona
neuralynx
neurologger
neuropixels


`ElectrodeConfig` is a bit more complex, referring to the exact configurations of groupings of channels found in a probe that the recording occured for. We will revisit this table and other related tables later in this notebook.

#### Question 6: Pick a mouse and find all recordings performed on that mouse.

## Tracking data

In contrast to the `Recording` table, `Tracking` table contains information about a particular tracking data stream found inside a session.

In [26]:
tracking.Tracking().describe();

-> acquisition.Session
tracking_time        : datetime                     # start time of this tracking recording
---
-> tracking.TrackingType
tracking_name        : varchar(40)                  # user-assign name of this tracking (e.g. 27032019laserSess1)
tracking_sample_rate : float                        # (Hz)
tracking_timestamps  : longblob                     # (s) timestamps of the frame in the tracking system clock, w.r.t the start of this tracking



In [27]:
tracking.Tracking()

animal_id,datasource_id,session_time start time of this session,tracking_time start time of this tracking recording,tracking_type,tracking_name user-assign name of this tracking (e.g. 27032019laserSess1),tracking_sample_rate (Hz),"tracking_timestamps (s) timestamps of the frame in the tracking system clock, w.r.t the start of this tracking"
00342c6d86e97d43,0,2019-07-04 20:35:12,2019-07-04 20:34:48,optitrack,2019-07-04_20-33-01,121.587,=BLOB=
40bfa03a7a7018cc,0,2019-09-17 19:43:06,2019-09-17 19:42:39,optitrack,2019-09-17_19-41-14,120.0,=BLOB=
61fd2ac184c13c73,0,2017-10-02 15:54:25,2017-10-02 15:54:25,axona,02102017s1,50.0,=BLOB=
61fd2ac184c13c73,0,2017-10-02 15:54:25,2017-10-02 16:28:38,axona,02102017laserSess1(1mw),50.0,=BLOB=
61fd2ac184c13c73,0,2017-10-02 15:54:25,2017-10-02 16:30:46,axona,02102017laserSess1(2mw),50.0,=BLOB=
61fd2ac184c13c73,0,2017-10-02 15:54:25,2017-10-02 16:32:47,axona,02102017laserSess1(3mw),50.0,=BLOB=
61fd2ac184c13c73,0,2017-10-02 15:54:25,2017-10-02 16:34:32,axona,02102017laserSess1(4mw),50.0,=BLOB=
61fd2ac184c13c73,0,2017-10-02 15:54:25,2017-10-02 16:37:12,axona,02102017laserSess1(7mw),50.0,=BLOB=
61fd2ac184c13c73,0,2017-10-02 15:54:25,2017-10-02 16:38:46,axona,02102017laserSess1(9mw),50.0,=BLOB=
61fd2ac184c13c73,0,2017-10-02 15:54:25,2017-10-02 16:42:03,axona,02102017laserSess1(12mw),50.0,=BLOB=


Again, `Tracking` *refers* to `Session` but also has it's own primary key attribute `tracking_time`, allowing for more than one tracking to be present per session, as long as they are separated in time. 

### Where is the data?

While `Recording` and `Tracking` tables both represented data recordings, you may have noticed that we didn't see the actual data points present in the table. This is because these tables are used to describe information **about recording and tracking**, and the actual data are loaded and handled by downstream tables as we will see shortly.

## Clustering

While a stream of electrical activities in itself can be interesting (i.e. refer to `LFP` below), we are often interested in working with individual spikes, and thus want to perform spike detection/clustering on the raw signal to extract these spikes. The ephys pipeline is built so that it can express very complex configurations of clustering.

Namely, **arbitrary time segments from one or more recordings across one or more sessions** can be combined to be the target of clustering. Let's now take a look at how that is represented in the tables.

### Specification of clustering time windows

To allow for flexible combination of one or more sessions, `ClusterSessionGrouping` table is used.

<img src="../images/ephys_pipeline/cluster_group.png" alt="drawing" width="700"/>

In [28]:
acquisition.ClusterSessionGroup.describe();

group_name           : varchar(36)                  
---
group_desc=""        : varchar(128)                 



`ClusterSessionGroup` table simply defines a group of one or more sessions, and the `ClusterSessionGroup.GroupMember` ties the member sessions to the group it belongs to.

Taking a look at an example group.

In [29]:
keys = acquisition.ClusterSessionGroup().fetch('KEY')

In [30]:
acquisition.ClusterSessionGroup() & keys[0]

group_name,group_desc
det_0033b4a248738944d352d904dd830875,Default session grouping for animal 61fd2ac184c13c73 on 57152018 10:57:26


and members for this session group

In [31]:
acquisition.ClusterSessionGroup.GroupMember & keys[3]

group_name,animal_id,datasource_id,session_time start time of this session
det_0fb019330f04214e309f896ab52f407e,61fd2ac184c13c73,0,2017-12-10 15:23:02


### Specifying time windows

<img src="../images/ephys_pipeline/group_windows.png" alt="drawing" width="900"/>

Once a group of one or more sessions is defined, `ClusterTimeWindows` and `ClusterTimeWindows.TimeWindow` are  used to subselect one or more time windows within the **concatenated session data**.

In [32]:
acquisition.ClusterTimeWindows.TimeWindow.describe();

-> acquisition.ClusterTimeWindows
window_start         : decimal(6,2)                 # (s) onset of this window, with respect to session onset
---
window_stop          : float                        # (s) offset of this window, with respect to session onset



For the most common scenario, the entire duration of the concatenated sessions are used, and this is indicated by `window_start = 0` and `window_stop = -1`.

In [33]:
acquisition.ClusterTimeWindows.TimeWindow()

group_name,time_window_id or maybe a hash of all the part windows,"window_start (s) onset of this window, with respect to session onset","window_stop (s) offset of this window, with respect to session onset"
det_0033b4a248738944d352d904dd830875,a1c6a0bfe97cd18966cb6fdb87c75687,0.00,-1.0
det_0646d5555bc1299437185e7d8c1c2816,a1c6a0bfe97cd18966cb6fdb87c75687,0.00,-1.0
det_0d7c5dd06d63c5f14395b914f7f803e7,a1c6a0bfe97cd18966cb6fdb87c75687,0.00,-1.0
det_0fb019330f04214e309f896ab52f407e,a1c6a0bfe97cd18966cb6fdb87c75687,0.00,-1.0
det_121ad375dd05ab19a4d212ee9678cc15,a1c6a0bfe97cd18966cb6fdb87c75687,0.00,-1.0
det_13391cf1a647f8e92ccedcb3afc2a251,a1c6a0bfe97cd18966cb6fdb87c75687,0.00,-1.0
det_144c54937ff40b02a50281270644fc2d,a1c6a0bfe97cd18966cb6fdb87c75687,0.00,-1.0
det_16ae3cc3249897cd2c53d66b39a90649,a1c6a0bfe97cd18966cb6fdb87c75687,0.00,-1.0
det_1ba81cdef70f9cf1d535152abe42af86,a1c6a0bfe97cd18966cb6fdb87c75687,0.00,-1.0
det_2e7d1e091f158b4cb1dc6918a3f1bade,a1c6a0bfe97cd18966cb6fdb87c75687,0.00,-1.0


# Organization of Clustering - ClusteringSetup and Clustering tables

The `ClusterSetup` table captures a valid combination of particular electrode probe (`ProbeInsertion`), clustering time windows across one or more concatenated Sessions (`ClusterTimeWindows`), as well as some clustering specific parameter (`ClusterParam`), and this depenencies are captured in the diagram below. 

<img src="../images/ephys_pipeline/clustering.png" alt="drawing" width="900"/>

While `ClusteringSetup` serves to specify the combination of clustering configurations, the `Clustering` table represents completed clustering.

Again, clusterings can be performed on:
+ Multiple concatenated sessions (although in most cases, just on one session)
+ Different time window(s) over the selected session(s)
+ Yielding units for this ***clustering***

`ClusteringSetup` is for users to specify:

+ the `ClusterSessionGroup` and `ClusterTimeWindows`
+ the probe this clustering is performed on - `ProbeInsertion`
+ the parameters to perform the clustering - `ClusterParam`
+ the directory to output this clustering results 
+ the electrode configuration information

Note: all of this is taken care of in the ingestion routine (more on that later)

`Clustering` - a processing step to either trigger the clustering or perform ingestion on performed clustering

### Curated clustering

Often times, an experimenter would follow an automatic clustering step and **manually curate** the clustering. In the pipeline, this is represented through `CuratedClustering` table.

<img src="../images/ephys_pipeline/curated_clustering.png" alt="drawing" width="350"/>

In [34]:
ephys.CuratedClustering.describe();

# A completed curation for a particular clustering
-> ephys.Clustering
curation_timestamps  : datetime                     # ensure complete datetime here to prevent collision (in the rare case of multiple people curate at the exact same datetime)
---
-> reference.Experimenter.proj(curator="experimenter")
curation_notes=""    : varchar(128)                 
-> reference.Repository
curation_output_dir  : varchar(128)                 



**CuratedClustering** captures the following information:
+ the curator
+ the time of curation
+ the directory of the new curation results

All downstream analysis are then performed on the `CuratedClustering`, allowing for different curated results to be compared side by side.

# Unit & Spike times

From the `CuratedClustering` results, definition of identified `Units`, their representative `Waveforms` as well as their spike times (`UnitSpikeTimes`) are extracted and captured in the corresponding table. Notice that the `UnitSpikeTimes` are matched back to the particular recording file during which the identified unit's spike occured.

<img src="../images/ephys_pipeline/unit_spikes.png" alt="drawing" width="500"/>

In [35]:
ephys.UnitSpikeTimes()

animal_id,datasource_id,insertion_time When this probe was inserted,group_name,time_window_id or maybe a hash of all the part windows,cluster_method name of clustering method,"cluster_param_name some unique name, or maybe hash of the dict of the param (ordered)","probe_model nick name, or other user-friendly model name of this probe",electrode_config_id hash of the group and group_member (ensure uniqueness),sync_method sync method name,curation_timestamps ensure complete datetime here to prevent collision (in the rare case of multiple people curate at the exact same datetime),unit,session_time start time of this session,recording_time start time of this recording,mean_firing_rate mean firing rate,spike_counts how many spikes in this recording of this unit,"unit_spike_times (s) spike times of this unit, relative to the start of the session this recording belongs to"
40bfa03a7a7018cc,0,2019-09-11 14:08:45,det_144c54937ff40b02a50281270644fc2d,a1c6a0bfe97cd18966cb6fdb87c75687,kilosort,default_kilosort,neuropixels_1.0,3bae3f7153f0305d02232efd41217754,linear_interp,2019-09-18 14:57:28,0,2019-09-17 19:43:06,2019-09-17 19:43:06,2.12436,16288,=BLOB=
40bfa03a7a7018cc,0,2019-09-11 14:08:45,det_144c54937ff40b02a50281270644fc2d,a1c6a0bfe97cd18966cb6fdb87c75687,kilosort,default_kilosort,neuropixels_1.0,3bae3f7153f0305d02232efd41217754,linear_interp,2019-09-18 14:57:28,1,2019-09-17 19:43:06,2019-09-17 19:43:06,4.88728,37472,=BLOB=
40bfa03a7a7018cc,0,2019-09-11 14:08:45,det_144c54937ff40b02a50281270644fc2d,a1c6a0bfe97cd18966cb6fdb87c75687,kilosort,default_kilosort,neuropixels_1.0,3bae3f7153f0305d02232efd41217754,linear_interp,2019-09-18 14:57:28,2,2019-09-17 19:43:06,2019-09-17 19:43:06,3.87792,29733,=BLOB=
40bfa03a7a7018cc,0,2019-09-11 14:08:45,det_144c54937ff40b02a50281270644fc2d,a1c6a0bfe97cd18966cb6fdb87c75687,kilosort,default_kilosort,neuropixels_1.0,3bae3f7153f0305d02232efd41217754,linear_interp,2019-09-18 14:57:28,3,2019-09-17 19:43:06,2019-09-17 19:43:06,1.21243,9296,=BLOB=
40bfa03a7a7018cc,0,2019-09-11 14:08:45,det_144c54937ff40b02a50281270644fc2d,a1c6a0bfe97cd18966cb6fdb87c75687,kilosort,default_kilosort,neuropixels_1.0,3bae3f7153f0305d02232efd41217754,linear_interp,2019-09-18 14:57:28,4,2019-09-17 19:43:06,2019-09-17 19:43:06,0.283544,2174,=BLOB=
40bfa03a7a7018cc,0,2019-09-11 14:08:45,det_144c54937ff40b02a50281270644fc2d,a1c6a0bfe97cd18966cb6fdb87c75687,kilosort,default_kilosort,neuropixels_1.0,3bae3f7153f0305d02232efd41217754,linear_interp,2019-09-18 14:57:28,5,2019-09-17 19:43:06,2019-09-17 19:43:06,0.618866,4745,=BLOB=
40bfa03a7a7018cc,0,2019-09-11 14:08:45,det_144c54937ff40b02a50281270644fc2d,a1c6a0bfe97cd18966cb6fdb87c75687,kilosort,default_kilosort,neuropixels_1.0,3bae3f7153f0305d02232efd41217754,linear_interp,2019-09-18 14:57:28,6,2019-09-17 19:43:06,2019-09-17 19:43:06,0.0164335,126,=BLOB=
40bfa03a7a7018cc,0,2019-09-11 14:08:45,det_144c54937ff40b02a50281270644fc2d,a1c6a0bfe97cd18966cb6fdb87c75687,kilosort,default_kilosort,neuropixels_1.0,3bae3f7153f0305d02232efd41217754,linear_interp,2019-09-18 14:57:28,7,2019-09-17 19:43:06,2019-09-17 19:43:06,2.39604,18371,=BLOB=
40bfa03a7a7018cc,0,2019-09-11 14:08:45,det_144c54937ff40b02a50281270644fc2d,a1c6a0bfe97cd18966cb6fdb87c75687,kilosort,default_kilosort,neuropixels_1.0,3bae3f7153f0305d02232efd41217754,linear_interp,2019-09-18 14:57:28,8,2019-09-17 19:43:06,2019-09-17 19:43:06,0.304412,2334,=BLOB=
40bfa03a7a7018cc,0,2019-09-11 14:08:45,det_144c54937ff40b02a50281270644fc2d,a1c6a0bfe97cd18966cb6fdb87c75687,kilosort,default_kilosort,neuropixels_1.0,3bae3f7153f0305d02232efd41217754,linear_interp,2019-09-18 14:57:28,9,2019-09-17 19:43:06,2019-09-17 19:43:06,0.308194,2363,=BLOB=


#### Question 7: Find all animals with at least 100,000 spike counts identified in a recording

## Summary diagram of the clustering structure
![](../images/clustering_detail_erd.png)

# Tracking

<img src="../images/ephys_pipeline/process_track.png" alt="drawing" width="900"/>

# Toward the results - Spikes and Tracking

<img src="../images/ephys_pipeline/task_spikes.png" alt="drawing" width="600"/>

***UnitSpikeTimes*** and ***Tracking*** data can be matched together - resulting in the ***SpikesTracking***:
> The spike times and tracking data (e.g. position, speed, head angle, etc.) associated with each spike, per unit

And such ***SpikesTracking*** can then be further narrowed down to the task, in ***TaskSpikesTracking***

In [36]:
analysis.TaskSpikesTracking()

animal_id,datasource_id,session_time start time of this session,task_type,task_start (s) timestamp of task onset with respect to the start of the session,unit unit id,spike_tracking_hash hash of the SpikesTracking primary_key,cell_selection_params_name user-friendly name,spike_times (s) task-related spike times (time with respect to the start of the task),speed (cm/s) task-related speed at each spike times,x_pos (cm) task-related x-pos at each spike times,y_pos (cm) task-related y-pos at each spike times,z_pos (cm) task-related z-pos at each spike times,head_yaw (degree) task-related head yaw (left/right angle) at each spike times,head_pitch (degree) task-related head pitch (up/down) at each spike times,head_roll (degree) task-related head roll (clockwise/anticlockwise) at each spike times,angular_speed (degree/s) task-related head angular speed at each spike times
40bfa03a7a7018cc,0,2019-09-17 19:43:06,OpenField,45.00,0,31ca6aaae907eb1fcc2ba953af199766,default,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=
40bfa03a7a7018cc,0,2019-09-17 19:43:06,OpenField,45.00,2,963d42326e666ba4fb9ee8dacfdbef6b,default,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=
40bfa03a7a7018cc,0,2019-09-17 19:43:06,OpenField,45.00,3,0481db364aa0ffe4cfeb82d262c4055b,default,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=
40bfa03a7a7018cc,0,2019-09-17 19:43:06,OpenField,45.00,3,06cce38e29c82f1b6c1f16e218d7b1b4,default,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=
40bfa03a7a7018cc,0,2019-09-17 19:43:06,OpenField,45.00,4,6bf44729b64cb70a645c1fca53151cbc,default,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=
40bfa03a7a7018cc,0,2019-09-17 19:43:06,OpenField,45.00,4,bd180e2d9c82adcd3a99faafddba0661,default,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=
40bfa03a7a7018cc,0,2019-09-17 19:43:06,OpenField,45.00,5,b4ec2fe41796dc975cd4d7ca870a9ae2,default,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=
40bfa03a7a7018cc,0,2019-09-17 19:43:06,OpenField,45.00,6,3750f0a4b56fbb50d16ef2aa091b1777,default,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=
40bfa03a7a7018cc,0,2019-09-17 19:43:06,OpenField,45.00,7,f70c1a6103a2f3f3528e96f8e30ef03b,default,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=
40bfa03a7a7018cc,0,2019-09-17 19:43:06,OpenField,45.00,8,8c7a6e79808cb7a3a5e0521154d9ac86,default,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=,=BLOB=


# Electrode Configuration

#### Electrode configuration represents the electrode grouping used in a particular recording from a probe:

<img src="../images/ephys_pipeline/econfig.png" alt="drawing" width="600"/>

For ***neuropixels*** probe: 
+ 1 group - which 384 channels used

For ***tetrode array*** probe:
+ each tetrode is a group
+ which channel assigned to which tetrode

In [37]:
reference.ElectrodeConfig()

"probe_model nick name, or other user-friendly model name of this probe",electrode_config_id hash of the group and group_member (ensure uniqueness),electrode_config_name user friendly name
4-tetrode-array,222cc704a75a2aea7777cc0899d809fd,g_2345-4_chn
4-tetrode-array,657119cc9e911c2c8a536abce16264f0,g_5678-4_chn
4-tetrode-array,e1832bbcce7a2aa5fd12f97d7448f9be,g_6781-4_chn
4-tetrode-array,e9c6dac82a8bcb02825b0be07735a967,g_1234-4_chn
8-tetrode-array,00784804f372476162c7d90e94d8dc3d,g_12345678-4_chn
neuropixels_1.0,3bae3f7153f0305d02232efd41217754,g_0-384_chn


In [38]:
reference.ElectrodeConfig.ElectrodeGroup * reference.ElectrodeConfig & 'electrode_config_name = "g_5678-4_chn"'

"probe_model nick name, or other user-friendly model name of this probe",electrode_config_id hash of the group and group_member (ensure uniqueness),electrode_group electrode group,electrode_config_name user friendly name
4-tetrode-array,657119cc9e911c2c8a536abce16264f0,5,g_5678-4_chn
4-tetrode-array,657119cc9e911c2c8a536abce16264f0,6,g_5678-4_chn
4-tetrode-array,657119cc9e911c2c8a536abce16264f0,7,g_5678-4_chn
4-tetrode-array,657119cc9e911c2c8a536abce16264f0,8,g_5678-4_chn


In [39]:
reference.ElectrodeConfig.Electrode * reference.ElectrodeConfig & 'electrode_config_name = "g_5678-4_chn"'

"probe_model nick name, or other user-friendly model name of this probe",electrode_config_id hash of the group and group_member (ensure uniqueness),electrode_group electrode group,electrode electrode,electrode_config_name user friendly name
4-tetrode-array,657119cc9e911c2c8a536abce16264f0,5,0,g_5678-4_chn
4-tetrode-array,657119cc9e911c2c8a536abce16264f0,5,1,g_5678-4_chn
4-tetrode-array,657119cc9e911c2c8a536abce16264f0,5,2,g_5678-4_chn
4-tetrode-array,657119cc9e911c2c8a536abce16264f0,5,3,g_5678-4_chn
4-tetrode-array,657119cc9e911c2c8a536abce16264f0,6,4,g_5678-4_chn
4-tetrode-array,657119cc9e911c2c8a536abce16264f0,6,5,g_5678-4_chn
4-tetrode-array,657119cc9e911c2c8a536abce16264f0,6,6,g_5678-4_chn
4-tetrode-array,657119cc9e911c2c8a536abce16264f0,6,7,g_5678-4_chn
4-tetrode-array,657119cc9e911c2c8a536abce16264f0,7,8,g_5678-4_chn
4-tetrode-array,657119cc9e911c2c8a536abce16264f0,7,9,g_5678-4_chn


![](../images/ephys_pipeline/rec_clus.png)

# Probe Insertion Location 

![](../images/ephys_pipeline/probe_insert_loc.png)

In [40]:
acquisition.ProbeInsertion.InsertionLocation.describe();

-> acquisition.ProbeInsertion
---
-> reference.BrainArea
hemisphere           : enum('left','right')         
-> reference.SkullReference
ml_location=null     : float                        # (um) relative to the skull-reference
ap_location=null     : float                        # (um) relative to the skull-reference
dv_location=null     : float                        # (um) relative to the surface of the dura at the implant location
ml_angle=null        : float                        # (degree)
ap_angle=null        : float                        # (degree)



# Probe Adjustment

![](../images/ephys_pipeline/probe_adjust.png)

In [41]:
acquisition.ProbeAdjustment.describe();

-> acquisition.ProbeInsertion
adjustment_time      : datetime                     
---
-> [nullable] acquisition.Session
-> reference.Experimenter
estimated_depth_change=null : float                        # (um) positive: moving more dorsal (shallower), negative: moving more ventral (deeper)
adjustment           : varchar(128)                 # new depth (ideally)

